In [1]:
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler

from statsmodels.formula.api import ols



from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings 

warnings.filterwarnings("ignore")

%matplotlib inline

## Check all of the discrete variable

In [11]:
# import x and y

X = pd.read_csv('../raw/X.csv')
y = pd.read_csv('../raw/y.csv', header=None)
y.columns = ['price']
df = pd.read_csv("../raw/kc_house_data_train.csv", index_col=0)

In [17]:
# list discrete variables
df.drop(columns=['id'], inplace=True)
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
1,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
2,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
3,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
4,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050


In [18]:
discrete_features = [
    'bedrooms',
    'bathrooms',
    'floors'
]

ordinal_variables = [
    'condition',
    'grade',
]

In [20]:
# make ordinal and disrete data frames 
discrete = df[discrete_features]
ordinal = df[ordinal_variables]

In [22]:
discrete.describe()

,bedrooms,bathrooms,floors
count,17290.000000,17290.000000,17290.000000
mean,3.372470,2.111943,1.490312
std,0.939346,0.770476,0.538909
min,0.000000,0.000000,1.000000
25%,3.000000,1.500000,1.000000
50%,3.000000,2.250000,1.500000
75%,4.000000,2.500000,2.000000
max,33.000000,8.000000,3.500000


In [26]:
ordinal.describe()

,condition,grade
count,17290.000000,17290.000000
mean,3.408502,7.654425
std,0.651296,1.174718
min,1.000000,1.000000
25%,3.000000,7.000000
50%,3.000000,7.000000
75%,4.000000,8.000000
max,5.000000,13.000000


# Ordinal values have no outliers

In [27]:
discrete[discrete.bedrooms > 10]

,bedrooms,bathrooms,floors
2356,11,3.00,2.0
8597,33,1.75,1.0


In [29]:
X[X.bedrooms<1]

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
483,3374500520,20150429T000000,0,0.00,2460,8049,2.0,0,0,3,8,2460,0,1990,0,98031,47.4095,-122.168,2520,8050
1120,6306400140,20140612T000000,0,0.00,3064,4764,3.5,0,2,3,7,3064,0,1990,0,98102,47.6362,-122.322,2360,4000
2430,1222029077,20141029T000000,0,0.75,384,213444,1.0,0,0,3,4,384,0,2003,0,98070,47.4177,-122.491,1920,224341
3032,7849202190,20141223T000000,0,0.00,1470,4800,2.0,0,0,3,7,1470,0,1996,0,98065,47.5265,-121.828,1060,7200
3609,2569500210,20141117T000000,0,2.50,2290,8319,2.0,0,0,3,8,2290,0,1985,0,98042,47.3473,-122.151,2500,8751
5424,3918400017,20150205T000000,0,0.00,1470,979,3.0,0,2,3,8,1470,0,2006,0,98133,47.7145,-122.356,1470,1399
6409,1453602309,20140805T000000,0,1.50,1430,1650,3.0,0,0,3,7,1430,0,1999,0,98125,47.7222,-122.290,1430,1650
9286,2954400190,20140624T000000,0,0.00,4810,28008,2.0,0,0,3,12,4810,0,1990,0,98053,47.6642,-122.069,4740,35061
10067,3980300371,20140926T000000,0,0.00,290,20875,1.0,0,0,1,1,290,0,1963,0,98024,47.5308,-121.888,1620,22850
10074,2310060040,20140925T000000,0,2.50,1810,5669,2.0,0,0,3,7,1810,0,2003,0,98038,47.3493,-122.053,1810,5685


In [31]:
X.bedrooms.median()

3.0

In [32]:
X.bathrooms.median()

2.25

### Let's impute the median values for the bathrooms

In [35]:
X.bathrooms = X.bathrooms.replace(0, X.bathrooms.median())
X.bedrooms = X.bedrooms.replace(0, X.bedrooms.median())
# Replace the positive outlier
X.bedrooms = X.bedrooms.replace(33, X.bedrooms.median())

In [38]:
X.bedrooms.describe()

count    17290.000000
mean         3.372817
std          0.907639
min          1.000000
25%          3.000000
50%          3.000000
75%          4.000000
max         11.000000
Name: bedrooms, dtype: float64

In [39]:
X.to_csv('../cleanedData/X.imputed.csv')

## Check for size errors and outliers

In [41]:
X.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2591820310,20141006T000000,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
1,7974200820,20140821T000000,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
2,7701450110,20140815T000000,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
3,9522300010,20150331T000000,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
4,9510861140,20140714T000000,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050


In [46]:
X.sort_values('sqft_lot', ascending=False).iloc[:20,:]

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
718,1020069017,20150327T000000,4,1.00,1300,1651359,1.0,0,3,4,6,1300,0,1920,0,98022,47.2313,-122.023,2560,425581
11616,3326079016,20150504T000000,2,1.00,710,1164794,1.0,0,0,2,5,710,0,1915,0,98014,47.6888,-121.909,1680,16730
13460,2623069031,20140521T000000,5,3.25,3010,1074218,1.5,0,0,5,8,2010,1000,1931,0,98027,47.4564,-122.004,2450,68825
16243,2323089009,20150119T000000,4,3.50,4030,1024068,2.0,0,0,3,10,4030,0,2006,0,98045,47.4619,-121.744,1830,11700
13543,722069232,20140905T000000,4,3.25,3770,982998,2.0,0,0,3,10,3770,0,1992,0,98058,47.4140,-122.087,2290,37141
2296,3626079040,20140730T000000,2,3.00,2560,982278,1.0,0,0,3,8,2560,0,2004,0,98014,47.6955,-121.861,1620,40946
16269,2624089007,20150320T000000,2,2.50,3900,920423,2.0,0,0,3,12,3900,0,2009,0,98065,47.5371,-121.756,2720,411962
397,2724079090,20150105T000000,4,3.25,3920,881654,3.0,0,3,3,11,3920,0,2002,0,98024,47.5385,-121.896,2970,112384
4666,1125079111,20150415T000000,4,5.50,6530,871200,2.0,0,2,3,11,6530,0,2008,0,98014,47.6640,-121.878,1280,858132
2705,2522029039,20140929T000000,3,2.00,3650,843309,2.0,0,0,4,7,3650,0,1991,0,98070,47.3627,-122.496,1870,273992
